In [13]:
#!/bin/bash
!kaggle datasets download hassan06/nslkdd

Dataset URL: https://www.kaggle.com/datasets/hassan06/nslkdd
License(s): unknown
 36% 5.00M/13.9M [00:00<00:00, 29.4MB/s]
100% 13.9M/13.9M [00:00<00:00, 67.8MB/s]


In [14]:
!unzip nslkdd.zip

Archive:  nslkdd.zip
  inflating: KDDTest+.arff           
  inflating: KDDTest+.txt            
  inflating: KDDTest-21.arff         
  inflating: KDDTest-21.txt          
  inflating: KDDTest1.jpg            
  inflating: KDDTrain+.arff          
  inflating: KDDTrain+.txt           
  inflating: KDDTrain+_20Percent.arff  
  inflating: KDDTrain+_20Percent.txt  
  inflating: KDDTrain1.jpg           
  inflating: index.html              
  inflating: nsl-kdd/KDDTest+.arff   
  inflating: nsl-kdd/KDDTest+.txt    
  inflating: nsl-kdd/KDDTest-21.arff  
  inflating: nsl-kdd/KDDTest-21.txt  
  inflating: nsl-kdd/KDDTest1.jpg    
  inflating: nsl-kdd/KDDTrain+.arff  
  inflating: nsl-kdd/KDDTrain+.txt   
  inflating: nsl-kdd/KDDTrain+_20Percent.arff  
  inflating: nsl-kdd/KDDTrain+_20Percent.txt  
  inflating: nsl-kdd/KDDTrain1.jpg   
  inflating: nsl-kdd/index.html      


In [90]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder, StandardScaler

import os

In [5]:
!python --version

Python 3.10.12


In [6]:
kdd_train = "https://raw.githubusercontent.com/gcamfer/Anomaly-ReactionRL/master/datasets/NSL/KDDTrain%2B.txt"
kdd_test = "https://raw.githubusercontent.com/gcamfer/Anomaly-ReactionRL/master/datasets/NSL/KDDTest%2B.txt"

In [7]:
data = pd.read_csv(kdd_train, sep=",")

In [8]:
data

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125968,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125972 entries, 0 to 125971
Data columns (total 43 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   0         125972 non-null  int64  
 1   tcp       125972 non-null  object 
 2   ftp_data  125972 non-null  object 
 3   SF        125972 non-null  object 
 4   491       125972 non-null  int64  
 5   0.1       125972 non-null  int64  
 6   0.2       125972 non-null  int64  
 7   0.3       125972 non-null  int64  
 8   0.4       125972 non-null  int64  
 9   0.5       125972 non-null  int64  
 10  0.6       125972 non-null  int64  
 11  0.7       125972 non-null  int64  
 12  0.8       125972 non-null  int64  
 13  0.9       125972 non-null  int64  
 14  0.10      125972 non-null  int64  
 15  0.11      125972 non-null  int64  
 16  0.12      125972 non-null  int64  
 17  0.13      125972 non-null  int64  
 18  0.14      125972 non-null  int64  
 19  0.15      125972 non-null  int64  
 20  0.16

In [11]:
data.columns

Index(['0', 'tcp', 'ftp_data', 'SF', '491', '0.1', '0.2', '0.3', '0.4', '0.5',
       '0.6', '0.7', '0.8', '0.9', '0.10', '0.11', '0.12', '0.13', '0.14',
       '0.15', '0.16', '0.18', '2', '2.1', '0.00', '0.00.1', '0.00.2',
       '0.00.3', '1.00', '0.00.4', '0.00.5', '150', '25', '0.17', '0.03',
       '0.17.1', '0.00.6', '0.00.7', '0.00.8', '0.05', '0.00.9', 'normal',
       '20'],
      dtype='object')

In [45]:
!mkdir ./data

In [46]:
!mv ./KDD* ./data

In [18]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations'
,'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate'
,'srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','outcome','level'])

In [53]:
torch.save(columns, "cols.pt")

In [57]:
data = pd.read_csv("./data/KDDTrain+.txt", sep=",")

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125972 entries, 0 to 125971
Data columns (total 43 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   0         125972 non-null  int64  
 1   tcp       125972 non-null  object 
 2   ftp_data  125972 non-null  object 
 3   SF        125972 non-null  object 
 4   491       125972 non-null  int64  
 5   0.1       125972 non-null  int64  
 6   0.2       125972 non-null  int64  
 7   0.3       125972 non-null  int64  
 8   0.4       125972 non-null  int64  
 9   0.5       125972 non-null  int64  
 10  0.6       125972 non-null  int64  
 11  0.7       125972 non-null  int64  
 12  0.8       125972 non-null  int64  
 13  0.9       125972 non-null  int64  
 14  0.10      125972 non-null  int64  
 15  0.11      125972 non-null  int64  
 16  0.12      125972 non-null  int64  
 17  0.13      125972 non-null  int64  
 18  0.14      125972 non-null  int64  
 19  0.15      125972 non-null  int64  
 20  0.16

In [60]:
data.columns = columns

In [61]:
data.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [27]:
def get_uniques(col, data_df=data):
    return np.unique(data_df[col].values)

In [28]:
get_uniques("protocol_type")

array(['icmp', 'tcp', 'udp'], dtype=object)

In [29]:
get_uniques("service")

array(['IRC', 'X11', 'Z39_50', 'aol', 'auth', 'bgp', 'courier',
       'csnet_ns', 'ctf', 'daytime', 'discard', 'domain', 'domain_u',
       'echo', 'eco_i', 'ecr_i', 'efs', 'exec', 'finger', 'ftp',
       'ftp_data', 'gopher', 'harvest', 'hostnames', 'http', 'http_2784',
       'http_443', 'http_8001', 'imap4', 'iso_tsap', 'klogin', 'kshell',
       'ldap', 'link', 'login', 'mtp', 'name', 'netbios_dgm',
       'netbios_ns', 'netbios_ssn', 'netstat', 'nnsp', 'nntp', 'ntp_u',
       'other', 'pm_dump', 'pop_2', 'pop_3', 'printer', 'private',
       'red_i', 'remote_job', 'rje', 'shell', 'smtp', 'sql_net', 'ssh',
       'sunrpc', 'supdup', 'systat', 'telnet', 'tftp_u', 'tim_i', 'time',
       'urh_i', 'urp_i', 'uucp', 'uucp_path', 'vmnet', 'whois'],
      dtype=object)

In [30]:
get_uniques("flag")

array(['OTH', 'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S2', 'S3',
       'SF', 'SH'], dtype=object)

In [31]:
get_uniques("outcome")

array(['back', 'buffer_overflow', 'ftp_write', 'guess_passwd', 'imap',
       'ipsweep', 'land', 'loadmodule', 'multihop', 'neptune', 'nmap',
       'normal', 'perl', 'phf', 'pod', 'portsweep', 'rootkit', 'satan',
       'smurf', 'spy', 'teardrop', 'warezclient', 'warezmaster'],
      dtype=object)

In [32]:
get_uniques("outcome").shape

(23,)

In [165]:
class KDDataset(Dataset):

    def __init__(
        self,
        features: torch.Tensor,
        labels: torch.Tensor,
        name: str = "kdd_train",
        labels_dict: dict= None
    ):
        super(KDDataset, self).__init__()
        self.features = features
        self.labels = labels
        self.labels_dict = labels_dict
        self.name = name

    def save(self, save_dir: str):
        from pathlib import Path
        save_dir = Path(save_dir)

        if not save_dir.exists():
            raise ValueError(f"Directory {save_dir} does not exist")

        raws = save_dir / "raws"

        if not raws.exists():
            raws.mkdir()

        file_path = raws / f"{self.name}.pt"
        torch.save((
            self.features, self.labels, self.name, self.labels_dict
        ), file_path)

    @staticmethod
    def load(saved_dir, is_train = True):
        from pathlib import Path

        name = "kdd_train" if is_train else "kdd_test"

        saved_dir = Path(saved_dir)
        if not saved_dir.exists():
            raise ValueError(f"Directory {saved_dir} does not exist")
        raws = saved_dir / "raws"
        if not raws.exists():
            raise ValueError(f"Directory {raws} does not exist")
        file_path = raws / f"{name}.pt"

        features, labels, name, labels_dict = torch.load(file_path, weights_only=True)
        return KDDataset(features, labels, name, labels_dict)

    @staticmethod
    def load_all(saved_dir):
        train = KDDataset.load(saved_dir, is_train=True)
        test = KDDataset.load(saved_dir, is_train=False)
        return train, test

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx: int):
        return (self.features[idx], self.labels[idx])

In [166]:
class KDDataProcessor:

    def __init__(
        self,
        data_path: str, cols_path: str, name: str
    ):
        self.data_path = data_path
        self.cols_path = cols_path

        self.features: torch.Tensor = None
        self.labels: torch.Tensor = None
        self.name: str = name
        self.labels_dict: dict = None

        self.is_processed = False

    def process(self):

        df = pd.read_csv(self.data_path, sep=",")
        df.columns = torch.load(self.cols_path, weights_only=True)

        labels_col = df["outcome"].values

        df, labels = self.remove_labels_column(df)

        df = self.encode_categorial_vars(df=df)

        df_numpy = self.normalize_dataframe(df=df)
        self.is_processed = True

        self.features = torch.from_numpy(df_numpy).float()
        self.labels = torch.from_numpy(labels).long()

        return self

    def remove_labels_column(self, df):

        encode_label = LabelEncoder()

        labels = encode_label.fit_transform(df["outcome"])

        self.labels_dict = {idx: label for idx, label in enumerate(encode_label.classes_)}

        df.drop("outcome", axis=1, inplace=True)
        df.drop("level", axis=1, inplace=True)

        return df, labels

    def encode_categorial_vars(self, df: pd.DataFrame):

        label_encodes = {}
        for col in df.columns:
            if df[col].dtype == np.dtype('object'):
                label_enc = LabelEncoder()
                df[col] = label_enc.fit_transform(df[col])
                label_encodes[col] = label_enc

        return df

    def normalize_dataframe(self, df):
        df_np = df.to_numpy()

        mean, std = df_np.mean(axis=-1, keepdims=True), df_np.std(axis=-1, keepdims=True)
        df_np =  (df_np - mean)/(std + 1e-8)

        return df_np

    def to_kddatset(self):
        if not self.is_processed:
            raise ValueError("Dataset is not processed, run 'process' method")

        return KDDataset(
            self.features, self.labels, self.name, self.labels_dict
        )

In [173]:
train_processor = KDDataProcessor(data_path="./data/KDDTrain+.txt", cols_path="./cols.pt", name="kdd_train").process()
kdd_train = train_processor.to_kddatset()
kdd_train.save("./")

In [174]:
test_processor = KDDataProcessor(data_path="./data/KDDTest+.txt", cols_path="./cols.pt", name="kdd_test").process()
kdd_test = test_processor.to_kddatset()
kdd_test.save("./")

In [178]:
!zip -r ./raws.zip ./raws

  adding: raws/ (stored 0%)
  adding: raws/kdd_train.pt (deflated 18%)
  adding: raws/kdd_test.pt (deflated 18%)


In [ ]:
from drl_ids.datasets import KDDataset